In [ ]:
import os
import sys
sys.path.append('../python_packages_static')
import pandas as pd
import numpy as np
import geopandas as gp
import rasterio
from rasterio.plot import show
from rasterio.merge import merge
import flopy as fp
import matplotlib.pyplot as plt
import mfexport

## Export parent model dis and hds files using [flopy](https://github.com/modflowpy/flopy) and [mfexport](https://github.com/aleaf/modflow-export)
### Note this will not run correctly in this repo because the large parent model files are not tracked.
**Parent model files (by Zell and Sanford, 2020) are available here: https://water.usgs.gov/GIS/metadata/usgswrd/XML/zell2020_wrr.xml**

In [ ]:
parent_dir = '../source_data/Shapefiles/NGWM'

parent_models = [
    '0101_0102_0103_0105_MF6_SS_Unconfined_250', 
    '0104_0106_0107_0109_MF6_SS_Unconfined_250',
    '0108_0110_0202_0203_MF6_SS_Unconfined_250',
    '0204_0206_0209_MF6_SS_Unconfined_250',
    '0205_MF6_SS_Unconfined_250',
    '0206_0207_0208_MF6_SS_Unconfined_250'
]


simname = 'mfsim.nam'

In [ ]:
for model in parent_models:
    print('working on model: {}'.format(model))
    
    #  load model with flopy
    model_ws = os.path.join(parent_dir, model)
    sim = fp.mf6.MFSimulation.load(simname, 'mf6', sim_ws=model_ws)
    m = sim.get_model()
    
    #  read model upper left coords and proj4 string from 'usgs.model.reference' file
    file = os.path.join(model_ws, 'usgs.model.reference')
    f = open(file, 'r')
    file_content = f.readlines()
    f.close()
    xul = float(file_content[0][-15:-2])
    yul = float(file_content[1][-15:-2])
    proj_string = file_content[-1][6:]
    
    #  set mfexport model grid
    print('setting model grid\n')
    grid = mfexport.MFexportGrid(
        delr=m.dis.delr.array, 
        delc=m.dis.delc.array, # grid spacing in meters 
        xul=xul, 
        yul=yul, 
        proj_str=proj_string
        #proj_str='+datum=WGS84 +lat_0=23 +lat_1=29.5 +lat_2=45.5 +lon_0=-96 +no_defs +proj=aea +units=m +x_0=0 +y_0=0'
    )
    
    print('exporting model data')
    
    #  export dis package rasters 
    
    #  create root output directory
    outfile_root = '../parent_model/mfexport_outfiles/'
    if not os.path.exists(outfile_root):
        os.makedirs(outfile_root)
    
    #  create 'dir' package output directory for model in question
    dis_out_path = os.path.join(outfile_root, model, 'dis')
    if not os.path.exists(dis_out_path):
        os.makedirs(dis_out_path)
    mfexport.export(
        m, 
        grid, 
        'dis', 
        output_path=dis_out_path
    )
    
    # Export heads as shapefiles
    
    #  create 'hds' package output directory for model in question
    hds_pth = os.path.join(parent_dir, 'outputs.' + model, model + '.hds' )
    
    hds_out_path = os.path.join(outfile_root, model, 'hds')
    if not os.path.exists(hds_out_path):
        os.makedirs(hds_out_path)
    
    hdry_unknown = -9999
    hnflo_unknown = -9999
    
    mfexport.export_heads(hds_pth,
                          grid, 
                          hdry=hdry_unknown, # example shows hdry=m.upw.hdry, no upw package used?
                          hnflo=hnflo_unknown, # example shows hnflo=m.bas6.hnoflo, no bas6 package used?
                          kstpkper=(0, 0), # steady state one stress period
                          output_path=hds_out_path) 
    
    print('Model export complete\n')

### Check out parent model extents compared to neversink model

In [ ]:
outfile_root = '../parent_model/mfexport_outfiles'

parent_models = [
    '0101_0102_0103_0105_MF6_SS_Unconfined_250', 
    '0104_0106_0107_0109_MF6_SS_Unconfined_250',
    '0108_0110_0202_0203_MF6_SS_Unconfined_250',
    '0204_0206_0209_MF6_SS_Unconfined_250',
    '0205_MF6_SS_Unconfined_250',
    '0206_0207_0208_MF6_SS_Unconfined_250'
]


#input_raster = os.path.join(outfile_root, model_folder_3, 'dis/rasters/idomain_lay0.tif ')

model_boundary_path = '../sciencebase/Shapefiles/Extents/Model_Extent_HUC12.shp'

gdf = gp.read_file(model_boundary_path)
gdf = gdf.to_crs('+datum=WGS84 +lat_0=23 +lat_1=29.5 +lat_2=45.5 +lon_0=-96 +no_defs +proj=aea +units=m +x_0=0 +y_0=0')

for model in parent_models:
    
    #print('plotting idomain of model: {}'.format(model))
    #input_raster = os.path.join(outfile_root, model, 'dis/rasters/idomain_lay0.tif')
    
    print('plotting hds_lay0 of model: {}'.format(model))
    input_raster = os.path.join(outfile_root, model, 'hds/rasters/hds_lay0_per0_stp0.tif')
    fig, ax = plt.subplots(figsize=(8,8))

    with rasterio.open(input_raster) as src:
        meta = src.meta
        show(src, ax=ax)
          
    gdf.plot(ax=ax, edgecolor='red', facecolor='None')
    plt.show()    